In [1]:
import numpy as np
import mne
import matplotlib

In [53]:
class BCICompetition4Set2B(object):
    
    def __init__(self, filename, load_sensor_names=None, labels_filename=None, feedback=True):
       
        assert load_sensor_names is None
        self.__dict__.update(locals())
        del self.self
    
    
    def load(self):                            #main code
        cnt = self.extract_data()               #load file replace nan with nanmean, return file with events...
        events, artifact_trial_mask = self.extract_events(cnt)   
        cnt.info["events"] = events            #event  converted in Class1=left, Class2=right
        cnt.info["artifact_trial_mask"] = artifact_trial_mask  #get loccation of rejected  trails 
        return cnt
    
    def extract_data(self):
        
        raw_gdf = mne.io.read_raw_gdf(self.filename, stim_channel="auto")
        raw_gdf.load_data()
        data = raw_gdf.get_data()

        for i_chan in range(data.shape[0]):
            # first set to nan, than replace nans by nanmean.
            this_chan = data[i_chan]
            data[i_chan] = np.where(
                this_chan == np.min(this_chan), np.nan, this_chan
            )
            mask = np.isnan(data[i_chan])
            chan_mean = np.nanmean(data[i_chan])
            data[i_chan, mask] = chan_mean

        
        gdf_events = mne.events_from_annotations(raw_gdf)
        raw_gdf = mne.io.RawArray(data, raw_gdf.info, verbose="WARNING")
      
        raw_gdf.info["gdf_events"] = gdf_events
        return raw_gdf

    def extract_events(self, raw_gdf):
       
        
        events, name_to_code = raw_gdf.info["gdf_events"]

    
        if not ("783" in name_to_code): #checking for cue unkown
            train_set = True
        else:
            train_set = False
            
       
        if train_set:
            trial_codes = [name_to_code['769'],name_to_code['770']]  # the classes                
        else:
            trial_codes = [name_to_code['783']]  # Que unkown for  test cases.

        
        trial_mask = [ev_code in trial_codes for ev_code in events[:, 2]]
        trial_events = events[trial_mask]
        print("Got {:d} markers".format(len(trial_events)))
       
        #change the position of  the classes  to   1  and  2.
        trial_events[:, 2] = trial_events[:, 2] - name_to_code['768']   
        
        # now also create 0-1 vector for rejected trials
            
        trial_start_events = events[events[:, 2] == name_to_code['768']]
        assert len(trial_start_events) == len(trial_events)
        artifact_trial_mask = np.zeros(len(trial_events), dtype=np.uint8)
        artifact_events = events[events[:, 2] == 1]
        
        for artifact_time in artifact_events[:, 0]:
            i_trial = trial_start_events[:, 0].tolist().index(artifact_time)
            artifact_trial_mask[i_trial] = 1

        return trial_events, artifact_trial_mask
        
     

In [58]:
fname = 'b0103T.gdf'
raw = BCICompetition4Set2B(fname)
fianl = raw.load()

Extracting EDF parameters from C:\Users\adm\moeedtesting\b0103T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 469010  =      0.000 ...  1876.040 secs...


C:\Users\adm\anaconda3\envs\envpytorch\lib\site-packages\mne\io\edf\edf.py:1044: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-53-d0f28357ec0f>:19: RuntimeWarning: Highpass cutoff frequency 100.0 is greater than lowpass cutoff frequency 0.5, setting values to 0 and Nyquist.
  raw_gdf = mne.io.read_raw_gdf(self.filename, stim_channel="auto")


Used Annotations descriptions: ['1023', '1077', '1078', '1079', '1081', '276', '277', '32766', '768', '769', '770', '781']
Got 160 markers
